# NLP를 활용한 레시피 추천 챗봇 구현
**개요**
- Collection 
- Preprocessing
- EDA
- Embeddings
- Modeling
- Streamlit

## 4. Embeddings
**모델링 과정**
1. 사전 학습된 모델을 Sentence Transformer 를 통해 불러오기
2. 수집하고 기본적인 전처리를 거친 데이터를 위의 모델을 통해 임베딩 벡터로 변환한 파생변수 생성하기
3. input 으로 사용자가 재료를 포함한 문자열을 입력하면 문장을 벡터화하여 기존의 임베딩 벡터와 코사인 유사도를 구하는 함수
4. 코사인 유사도 상위 기준으로 n개를 output 으로 추천
5. OpenAI 의 API 를 활용하여 사용자의 문장 형태 input 의 의도를 파악하여 모델링 함수를 실행하도록 연동
6. streamlit 에 연동하여 배포


### 라이브러리 import

In [1]:
# 데이터 분석
import pandas as pd
import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 진행시간 표시
import swifter
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
# 파이토치
import torch

# 문장 임베딩, transformer 유틸리티
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import SentenceTransformer, models

In [3]:
# 객체 복사
import copy

# JSON 형식 데이터 처리
import json

# 데이터 수집
import requests

In [4]:
# 데이터베이스 활용
import sqlite3 
import pickle

In [5]:
# OpenAI API 활용
import openai 
import os # 운영체제
import sys # 파이썬 변수, 함수 엑세스 
from dotenv import load_dotenv # 환경 변수 로드(API Key 보안)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
load_dotenv()    
openai.api_key = os.getenv('OPENAI_API_KEY')

In [6]:
# 실행 os 확인
cur_os = sys.platform
cur_os

'darwin'

### 데이터 불러오기

In [8]:
df = pd.read_csv('data/preprocessed_recipes.csv')
df.shape

(4340, 10)

In [9]:
df.head(1)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,https://static.wtable.co.kr/image/production/s...,통깨 꼬시래기 식초 용냉면육수 매실청 쌈무 참기름 고춧가루 소금 오이 고추장 달걀,"['끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\r\n(tip...","꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ...",12


In [10]:
difficulty_map = {1: '쉬움', 2: '보통', 3: '어려움'}
df['난이도'] = df['난이도'].map(difficulty_map)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요리      4340 non-null   object
 1   종류      4340 non-null   object
 2   난이도     4340 non-null   object
 3   소요시간    4340 non-null   int64 
 4   링크      4340 non-null   object
 5   사진      4340 non-null   object
 6   재료      4340 non-null   object
 7   요리방법    4340 non-null   object
 8   설명      4340 non-null   object
 9   재료수     4340 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 339.2+ KB


### 전이학습을 통한 임베딩 벡터 생성
- 사전학습 모델을 활용
- feature 생성 및 비교

### feature 생성

In [12]:
df['feature'] = df['재료'] + df['요리'] + df['설명'] + df['종류'] + df['난이도'] + df['요리방법']
df['feature']

0       통깨 꼬시래기 식초 용냉면육수 매실청 쌈무 참기름 고춧가루 소금 오이 고추장 달걀꼬...
1       통깨 참기름 닭가슴살 찹쌀밥 다진당근 소금 물 통마늘닭죽아플 때나 기운이 없을 때 ...
2       통깨 양조식초 밥 깻잎 참기름 다진마늘 새싹 설탕 상추 양파 고춧가루 간장 당근 고...
3       후춧가루 올리고당 땅콩가루 굴소스 페페론치노 다진마늘 돼지등갈비 통후추 생강가루 설...
4       후춧가루 파스타면 케이퍼 마늘 루꼴라 레몬제스트 소금 앤초비 올리브오일앤초비 파스타...
                              ...                        
4335    후춧가루 대파 청양고추 다진마늘 설탕 홍고추 신김치 멸치다시마육수 고춧가루 김치국물...
4336    후춧가루 통깨 대파 밥 닭다리살 굴소스 올리고당 꽈리고추 다진마늘 참기름 생강가루 ...
4337    후춧가루 녹인버터 체다슬라이스치즈 베이컨 소금 감자 플레인요거트 파슬리가루 올리브오...
4338    생강가루 전분가루 소금 케첩 순살고등어 땅콩 맛술 간장 물 고추장 식용유 후춧가루 ...
4339    후춧가루 디종머스타드 파슬리 대파 코코넛밀크 소금 타임 두유 뉴트리셔널이스트대파그라...
Name: feature, Length: 4340, dtype: object

In [13]:
df.to_csv('data/feature_recipes.csv', index=False)

### 사전 학습 모델
- model : Sentence Transformer
- pre trained model : `jhgan/ko-sroberta-multitask`

In [14]:
model_name = 'jhgan/ko-sroberta-multitask'

In [15]:
model = SentenceTransformer(model_name)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [16]:
# feature1
feature_name = 'feature'
df[f'{model_name}-' + f'{feature_name}'] = df['feature'].progress_apply(lambda x: model.encode(x))

  0%|          | 0/4340 [00:00<?, ?it/s]

In [18]:
df.columns = ['요리', '종류', '난이도', '소요시간', '링크', '사진', '재료', '요리방법', '설명', '재료수',
       'feature', 'ko-sroberta-multitask-feature']

In [19]:
df = df[['요리', '종류', '난이도', '재료', '요리방법', '설명', '재료수', '소요시간', '링크', '사진', 
         'feature', 'ko-sroberta-multitask-feature']]
df

,요리,종류,난이도,재료,요리방법,설명,재료수,소요시간,링크,사진,feature,ko-sroberta-multitask-feature
0,꼬시래기 물냉면,메인요리,쉬움,통깨 꼬시래기 식초 용냉면육수 매실청 쌈무 참기름 고춧가루 소금 오이 고추장 달걀,"['끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\r\n(tip...","꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ...",12,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,https://static.wtable.co.kr/image/production/s...,통깨 꼬시래기 식초 용냉면육수 매실청 쌈무 참기름 고춧가루 소금 오이 고추장 달걀꼬...,"[0.21892446, -0.08537187, -0.08634098, -0.2353..."
1,닭죽,메인요리,쉬움,통깨 참기름 닭가슴살 찹쌀밥 다진당근 소금 물 통마늘,"['냄비에 물과 닭가슴살, 마늘, 대파를 넣고 15분간 삶아주세요. ', '삶은 닭...",아플 때나 기운이 없을 때 간단히 끓여 먹기 좋은 닭죽! 맛이 좋은 건 물론 속을 ...,8,30,https://wtable.co.kr/recipes/QawUyVgt3e5wvCT9d...,https://static.wtable.co.kr/image/production/s...,통깨 참기름 닭가슴살 찹쌀밥 다진당근 소금 물 통마늘닭죽아플 때나 기운이 없을 때 ...,"[0.3989115, -0.18220846, 0.1402082, -0.3062960..."
2,생채소 참치비빔밥,메인요리,쉬움,통깨 양조식초 밥 깻잎 참기름 다진마늘 새싹 설탕 상추 양파 고춧가루 간장 당근 고...,"['참치는 꽉 짜서 기름을 빼주세요. ', '채소는 가볍게 씻어 채반에서 물기를 빼...",별다른 반찬이 없을 때 해 먹기 딱 좋은 생채소 참치비빔밥 레시피 소개할게요. 집에...,16,20,https://wtable.co.kr/recipes/VxEeT7YiZ5QQETGmj...,https://static.wtable.co.kr/image/production/s...,통깨 양조식초 밥 깻잎 참기름 다진마늘 새싹 설탕 상추 양파 고춧가루 간장 당근 고...,"[0.32626605, -0.10854334, 0.06457064, -0.14995..."
3,등갈비강정,메인요리,보통,후춧가루 올리고당 땅콩가루 굴소스 페페론치노 다진마늘 돼지등갈비 통후추 생강가루 설...,['등갈비는 찬물에 30분 정도 담가 핏물을 뺀 후 흐르는 물에 깨끗하게 씻어주세요...,돼지고기 등갈비는 야들야들한 살코기에 육즙이 가득 스며들어 뜯어 먹는 재미까지 있어...,18,40,https://wtable.co.kr/recipes/1nyadXLMLxWsNiL3y...,https://static.wtable.co.kr/image/production/s...,후춧가루 올리고당 땅콩가루 굴소스 페페론치노 다진마늘 돼지등갈비 통후추 생강가루 설...,"[0.28909966, -0.33386758, -0.2219468, -0.02882..."
4,앤초비 파스타,메인요리,쉬움,후춧가루 파스타면 케이퍼 마늘 루꼴라 레몬제스트 소금 앤초비 올리브오일,"['앤초비와 케이퍼는 작게 다지고 마늘은 편으로 썰어주세요.', '냄비에 물과 소금...",앤초비는 지중해나 유럽 근해에서 나는 멸치류의 작은 물고기를 발효시킨 젓갈입니다. ...,9,20,https://wtable.co.kr/recipes/PJwQfoHitXrpY8n3B...,https://static.wtable.co.kr/image/production/s...,후춧가루 파스타면 케이퍼 마늘 루꼴라 레몬제스트 소금 앤초비 올리브오일앤초비 파스타...,"[0.6808941, 0.37798557, -0.5026342, -0.0001627..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4335,돼지고기 김치찜,오븐 요리,보통,후춧가루 대파 청양고추 다진마늘 설탕 홍고추 신김치 멸치다시마육수 고춧가루 김치국물...,['김치는 속 재료를 털어내고 꼭지를 붙인 채 3등분하여 썰어주세요. 볼에 양념장 ...,부드럽게 익은 묵은지에 삼겹살을 함께 먹으면 환상의 궁합이죠! 남녀노소 모두 좋아하...,12,60,https://wtable.co.kr/recipes/VLWSSwqgVZWJeeCmH...,https://static.wtable.co.kr/image/production/s...,후춧가루 대파 청양고추 다진마늘 설탕 홍고추 신김치 멸치다시마육수 고춧가루 김치국물...,"[0.1760793, -0.04701886, 0.12539494, -0.059310..."
4336,닭고기 덮밥,오븐 요리,쉬움,후춧가루 통깨 대파 밥 닭다리살 굴소스 올리고당 꽈리고추 다진마늘 참기름 생강가루 ...,"['닭 다리 살은 한 입 크기로 썰어준 후 밑간 재료에 10분간 재워주세요.', '...",쫄깃한 닭다리살과 은은하게 매운 꽈리고추가 정말 잘 어울리는 덮밥. 간편하고 근사한...,16,40,https://wtable.co.kr/recipes/4okDDgv6FXaWVrdjE...,https://static.wtable.co.kr/image/production/s...,후춧가루 통깨 대파 밥 닭다리살 굴소스 올리고당 꽈리고추 다진마늘 참기름 생강가루 ...,"[0.08764206, 0.08066006, 0.1789572, -0.0422857..."
4337,감자 버터 오븐구이,오븐 요리,보통,후춧가루 녹인버터 체다슬라이스치즈 베이컨 소금 감자 플레인요거트 파슬리가루 올리브오일,['껍질째 깨끗이 씻은 감자를 양쪽에 나무젓가락을 놓고 일정한 간격으로 얇게 칼집을...,"맛있을 수밖에 없는 메뉴! 담백한 감자에 짭조름한 치즈, 베이컨까지 어우러진 감자 ...",9,60,https://wtable.co.kr/recipes/isqPQBYFeGTEtbKBe...,https://static.wtable.co.kr/image/production/s...,후춧가루 녹인버터 체다슬라이스치즈 베이컨 소금 감자 플레인요거트 파슬리가루 올리브오...,"[-0.061336264, -0.14253633, 0.50772816, 0.0692..."
4338,고등어 강정,오븐 요리,보통,생강가루 전분가루 소금 케첩 순살고등어 땅콩 맛술 간장 물 고추장 식용유 후춧가루 ...,"['순살 고등어는 한입 크기로 잘라 밑간 재료에 10분간 재워주세요.', '파채는 ...","고등어를 색다르게 즐기는 요리. 비린 맛이 하나도 없고, 매콤 달콤한 소스와 너무 ...",19,30,https://wtable.co.kr/recipes/KeZoBFZT8b2iHPzfv...,https://static.wtable.co.kr/image/production/s...,생강가루 전분가루 소금 케첩 순살고등어 땅콩 맛술 간장 물 고추장 식용유 후춧가루 ...,"[0.32755455, 0.25685662, 0.5167652, 0.6005948,..."


In [20]:
df.to_pickle('data/compact_kosroberta_recipes.pkl')
df = pd.read_pickle('data/compact_kosroberta_recipes.pkl')

In [22]:
df.head(1)

,요리,종류,난이도,재료,요리방법,설명,재료수,소요시간,링크,사진,feature,ko-sroberta-multitask-feature
0,꼬시래기 물냉면,메인요리,쉬움,통깨 꼬시래기 식초 용냉면육수 매실청 쌈무 참기름 고춧가루 소금 오이 고추장 달걀,"['끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\r\n(tip...","꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ...",12,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,https://static.wtable.co.kr/image/production/s...,통깨 꼬시래기 식초 용냉면육수 매실청 쌈무 참기름 고춧가루 소금 오이 고추장 달걀꼬...,"[0.21892446, -0.08537187, -0.08634098, -0.2353..."


## DB에 임베딩 벡터 저장하고, 불러오는 과정
- 위에서 적용한 pickle 형식으로 데이터를 저장하는 것이 더 효율적이라고 판단됨

In [23]:
# # 데이터 프레임의 embedding
# def save_embeddings_to_db(df, db_file, model):
#     conn = sqlite3.connect(db_file)
#     cursor = conn.cursor()
#     cursor.execute('''CREATE TABLE IF NOT EXISTS embedding_table (id INTEGER PRIMARY KEY, embedding_data BLOB)''')
#     conn.commit()

#     for index, row in df.iterrows():
#         text = row['feature']
#         embedding = model.encode(text)
#         embedding_binary = pickle.dumps(embedding)
#         cursor.execute('''INSERT INTO embedding_table (id, embedding_data) VALUES (?,?)''', (index, embedding_binary))
#         conn.commit()

#     conn.close()

In [24]:
# save_embeddings_to_db(df, 'embedding.db', model)

In [25]:
# def add_embeddings_to_df(df, db_file):
#     conn = sqlite3.connect(db_file)
#     cursor = conn.cursor()

#     # 데이터베이스에서 임베딩 정보 불러오기
#     cursor.execute('''SELECT id, embedding_data FROM embedding_table''')
#     rows = cursor.fetchall()
    
#     # 불러온 임베딩 정보를 딕셔너리로 변환
#     embeddings_dict = {}
#     for row in rows:
#         index = row[0]
#         embedding_binary = row[1]
#         embedding = pickle.loads(embedding_binary)
#         embeddings_dict[index] = embedding
    
#     # 데이터프레임에 임베딩 정보 추가
#     embeddings = []
#     for index, row in df.iterrows():
#         if index in embeddings_dict:
#             embedding = embeddings_dict[index]
#             embeddings.append(embedding)
#         else:
#             embeddings.append(None)
#     df['embeddings'] = embeddings
    
#     conn.close()
#     return df


In [26]:
# add_embeddings_to_df(df, 'embedding.db')
# df